<a href="https://colab.research.google.com/github/Alexrosulek/Cs50/blob/main/makearticle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

import os

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})
!nohup ollama serve &



!ollama pull gemma3:1b
!ollama pull gemma3:4b
!pip install ollama
!pip install crawl4ai
!pip install aiohttp
!pip install pillow
!pip install beautifulsoup4
!pip install wikipedia
!pip install googlesearch-python
!pip install playwright
!playwright install chromium
!nohup ollama serve &

import asyncio
from crawl4ai import AsyncWebCrawler
from crawl4ai.async_configs import BrowserConfig, CrawlerRunConfig
from urllib.parse import urlparse
import asyncio
import random
import time
import aiohttp
import shutil
import os

import os
from PIL import Image
import base64
import io
import subprocess
import requests
from bs4 import BeautifulSoup
import wikipedia

import textwrap

import re
import json
from googlesearch import search

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  16888      0 --:--:-- --:--:-- --:--:-- 16875
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,628 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repo

In [ ]:


folder = "downloaded_images"
os.makedirs(folder, exist_ok=True)

def is_valid_phone(value: str) -> bool:
    print(value)
    return bool(re.fullmatch(r"\d{3}-\d{3}-\d{4}", value.strip()))

def is_valid_email(value: str) -> bool:
    print(value)
    return bool(re.fullmatch(r"[^@\s]+@[^@\s]+\.[a-zA-Z0-9]+", value.strip()))

def is_valid_url(value: str) -> bool:
    print(value)
    return value.strip().startswith("http")

def is_valid_json(value: str) -> bool:
    print(value)
    try:
        json.loads(value)
        return True
    except Exception:
        return False

def is_non_empty_string(value: str) -> bool:
    print(value)
    return isinstance(value, str) and len(value.strip()) > 0

def is_valid_dict(value) -> bool:
    print(value)
    return isinstance(value, dict)
FIELD_VALIDATORS = {
    "extract_phone": is_valid_phone,
    "extract_email": is_valid_email,
    "extract_website": is_valid_url,
    "extract_operating_hours": is_valid_dict,
    "extract_holiday_hours": is_valid_dict,
    "extract_categories": is_valid_json,
    "extract_delivery_services": is_valid_json,
    "extract_social_media": is_valid_json,
    "extract_stocked_brands": is_valid_json,
    "extract_inventory_categories": is_valid_json,
    "extract_customer_reviews": is_valid_json,
    "extract_admission": is_non_empty_string,
    "extract_date_available": is_non_empty_string,
    "extract_attendance_amount": is_non_empty_string,
    "extract_exhibitor_amount": is_non_empty_string
}
FIELD_EXTRACTORS = {
        "extract_phone": 'From the content, extract the phone number in this format: "727-123-4567"\nOnly return the phone string, no extra text, prefixes, marks, or quotes, only the phone number itself.',
        "extract_email": 'From the content, extract the email address in this format: "name@example.com"\nOnly return the email string, no extra text, prefixes,  marks, or quotes, only the email itself.',
        "extract_website": 'From the content, extract the website URL in this format: "https://example.com"\nOnly return the URL string, no extra text, prefixes,  marks, or quotes, only the website url itself with https://.',
        "extract_operating_hours": """From the content, extract operating hours in this format, no extra text, prefixes, marks or quotes, only the dictionary itself:
{
  "monday": "9:00 AM - 5:00 PM",
  "tuesday": "9:00 AM - 5:00 PM",
  "wednesday": "9:00 AM - 5:00 PM",
  "thursday": "9:00 AM - 5:00 PM",
  "friday": "9:00 AM - 5:00 PM",
  "saturday": "Closed",
  "sunday": "Closed"
}
Only return DICTIONARY, no extra text. no `, nothing before or after dictionary, ONLY VALID DICTIONARY.""",
        "extract_holiday_hours": "From the content, extract holiday hours in this format: {'2024-12-25': 'Closed', '2024-12-31': '10:00 AM - 4:00 PM'}\nOnly return DICTIONARY, no extra text, prefixes,  marks or quotes, only the dictionary itself. no `, nothing before or after dictionary, ONLY VALID DICTIONARY.",
        "extract_categories": "From the content, extract shop categories in this format: ['Nonprofit', 'Holy']\nOnly return JSON, no extra text, prefixes,  marks or quotes, only the json itself. no `, nothing before or after json, ONLY VALID JSON.",
        "extract_delivery_services": "From the content, extract delivery services in this format: ['Uber Eats', 'Self Delivery']\nOnly return JSON, no extra text, prefixes,  marks or quotes, only the json itself.  no `, nothing before or after json, ONLY VALID JSON.",
        "extract_social_media": "From the content, extract social media links in this format: {'facebook': 'url', 'instagram': 'url'}\nOnly return JSON, no extra text, prefixes,  marks or quotes, only the json itself. no `, nothing before or after json, ONLY VALID JSON.",
        "extract_stocked_brands": "From the content, extract stocked brands in this format: ['Nike', 'Adidas']\nOnly return JSON, no extra text, prefixes,  marks or quotes, only the json itself. no `, nothing before or after json, ONLY VALID JSON.",
        "extract_inventory_categories": "From the content, extract inventory categories in this format: {'Apparel': ['Shirts', 'Hoodies']}\nOnly return JSON, no extra text, prefixes,  marks or quotes, only the json itself.  no `, nothing before or after json, ONLY VALID JSON.",
        "extract_customer_reviews": "From the content, extract customer reviews in this format: [{'user'': 'John', 'comment': 'Great!', 'rating': 5}]\nOnly return JSON, no extra text, prefixes,  marks or quotes, only the json itself. no `, nothing before or after json, ONLY VALID JSON.",
        "extract_admission": "From the content, extract the admission info as a string. Only return the string, no extra text, prefixes, marks or quotes, only the string admission itself.  no `, nothing before or after json, ONLY VALID JSON.",
        "extract_date_available": "From the content, extract date available as a string. Only return the string, no extra text, prefixes, marks or quotes, only the string date available itself. no `, nothing before or after json, ONLY VALID JSON.",
        "extract_attendance_amount": "From the content, extract attendance amount as a string. Only return the string, no extra text, prefixes, marks or quotes, only the string attendence amount itself. no `, nothing before or after json, ONLY VALID JSON.",
        "extract_exhibitor_amount": "From the content, extract exhibitor amount as a string. Only return the string, no extra text, prefixes, marks or quotes, only the string exhibitor amount itself. no `, nothing before or after json, ONLY VALID JSON. "
    }
import random
import wikipedia

async def handle_wikipedia(task):
    try:
        name = task['target'].get("name")
        city = task['target'].get("city", "")
        query = f"{name} {city}".strip()
        print(f"📚 Wikipedia lookup for: {query}")

        page = wikipedia.page(query, auto_suggest=True)
        content = page.content

        # If short, return all
        if len(content) <= 2000:
            return content

        # Otherwise, chunk and pick 4 from middle
        chunk_size = 500
        chunks = [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]

        if len(chunks) <= 6:
            selected_chunks = chunks  # Not enough to skip ends
        else:
            middle_chunks = chunks[2:-2]
            selected_chunks = random.sample(middle_chunks, min(4, len(middle_chunks)))

        return "\n\n".join(selected_chunks)

    except Exception as e:
        print(f"Wikipedia fetch failed: {e}")
        return None


async def handle_google_search(task, crawler):
    try:
        target = task['target']
        query = f"{target.get('name', '')} {target.get('city', '')} {target.get('state', '')}".strip()
        print(query)
        valid_results = ''

        results = list(search(query))

        # Randomly shuffle and pick half
        half_results = random.sample(results, max(1, len(results) // 2))



        for url in half_results:
            try:
                res = requests.get(url, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
                soup = BeautifulSoup(res.text, "html.parser")

                title = soup.title.string.strip() if soup.title else "No title"
                meta_tag = soup.find("meta", {"name": "description"})
                description = meta_tag["content"].strip() if meta_tag else "No description"

                prompt = (
                    f"Is this page about the query '{query}'? Be very strict, no random things this is for a specific {target.get('name', '')}.\n\n"
                    f"Title: {title}\n"
                    f"Description: {description}\n\n"
                    f"Return true or false."
                )

                print("🔍 running ollama for relevance check")
                result = run_ollama1(prompt)
                if "true" in result.lower():
                    print(f"✅ Relevant: {url}")
                    content = await getsite(url, crawler)
                    if content and content.success:
                        markdown = content.markdown.fit_markdown
                        if len(markdown) <= 1500:
                            valid_results += markdown

                        # Chunk and pick 4 from the middle
                        chunk_size = 500
                        chunks = [markdown[i:i + chunk_size] for i in range(0, len(markdown), chunk_size)]
                        if len(chunks) <= 6:
                            selected_chunks = chunks
                        else:
                            selected_chunks = random.sample(chunks[2:-2], min(4, len(chunks[2:-2])))

                        valid_results += "\n\n".join(selected_chunks) + f"\n\n<- EXTRACTED FROM THIS URL: {url}\n\n"
            except Exception as inner_err:
                print(f"⚠️ Error processing {url}: {inner_err}")

        if valid_results != '':
            return valid_results
        else:
            return None
    except Exception as e:
        print(f"❌ Error in handle_google_search: {e}")
        return None

async def search2(task, crawler):
    print("getting google results")
    google_results = await handle_google_search(task, crawler)
    print("getting wiki results")
    wiki_results = await handle_wikipedia(task)

    imageandalt = None
    result = None

    print("done searching")
    mainstring = ''
    if google_results:
        print("got some google results")
        mainstring = google_results
        if wiki_results:
            print("got some wiki results")
            mainstring += wiki_results
    if result:
        if result.success:

            mainstring += result.markdown.fit_markdown
    if mainstring == '' or len(mainstring) < 500:
        if not imageandalt:
            return False, None, None
        return True, None, imageandalt
    return True, mainstring, imageandalt



def run_ollama4(prompt, model="gemma3:4b"):
    print(f"🧠 Running Ollama with model={model}")
    try:
        proc = subprocess.run(
            ["ollama", "run", model],
            input=prompt.encode("utf-8"),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=556
        )
        stdout = proc.stdout.decode("utf-8").strip()
        stderr = proc.stderr.decode("utf-8").strip()
        print(f"Ollama stdout:\n{stdout}")
        if stderr:
            print(f"Ollama stderr:\n{stderr}")
        return stdout
    except Exception as e:
        print(f"Ollama run failed: {e}")
        return ""
def run_ollama1(prompt, model="gemma3:1b"):
    print(f"🧠 Running Ollama with model={model}")
    try:
        proc = subprocess.run(
            ["ollama", "run", model],
            input=prompt.encode("utf-8"),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=356
        )
        stdout = proc.stdout.decode("utf-8").strip()
        stderr = proc.stderr.decode("utf-8").strip()
        print(f"Ollama stdout:\n{stdout}")
        return stdout
    except Exception as e:
        print(f"Ollama run failed: {e}")
        return ""


def normalize_and_validate_url(url: str) -> str | None:
    try:
        # Normalize: ensure https://
        if not url.startswith("https://"):
            if url.startswith("http://"):
                url = url.replace("http://", "https://", 1)
            else:
                url = "https://" + url.lstrip("/")

        # Validate
        parsed = urlparse(url)
        if not parsed.scheme or not parsed.netloc:
            return None  # Invalid URL

        return url
    except Exception as e:
        print(f"❌ Error normalizing/validating {url}: {e}")
        return None
class CrawlerManager:
    def __init__(self):
        self.crawler = None

    async def start(self):
        if self.crawler is None:
            self.crawler = AsyncWebCrawler(config=BrowserConfig())
            await self.crawler.__aenter__()

    async def stop(self):
        if self.crawler:
            await self.crawler.__aexit__(None, None, None)
            self.crawler = None

    async def crawl(self, url: str):
        if not self.crawler:
            raise RuntimeError("Crawler not started")

        url = normalize_and_validate_url(url)
        if not url:
            return None

        try:
            run_config = CrawlerRunConfig()
            result = await self.crawler.arun(url=url, config=run_config)
            return result
        except Exception as e:
            print(f"❌ crawl error for {url}: {e}")
            return None
async def getsite(url: str, crawler_mgr: CrawlerManager):
    return await crawler_mgr.crawl(url)






def summarize_large_text(prompt, large_text: str, max_length: int = 4000) -> str | None:
    try:
        # Split large text into chunks (~1024 chars each)
        chunks = textwrap.wrap(large_text, width=2000)
        print(len(chunks), 'got this many chunks')
        summaries = []
        for idx, chunk in enumerate(chunks):
            prompt = f"Extract {prompt} info. keep facts, keep details about {prompt}, keep operating hours ect, all details, REMOVE HTML AND JS, u are not doing anything else but extracting/consolidating:\n\n{chunk}"
            summary = run_ollama1(prompt)
            if summary:
                summaries.append(summary)
            else:
                print(f"⚠️ Failed to summarize chunk {idx}")

        # Aggregate all summaries
        combined = "".join(summaries)

        if len(combined) > max_length:
            print("summarizing again")
            chunks = textwrap.wrap(combined, width=2000)

            summaries = []
            for idx, chunk in enumerate(chunks):
                prompt = f"Extract {prompt} info concisely, based on {prompt}, keep operating hours phone website ect all details, u are not doing anything else but extracting/consolidating, return it back do not tell me how good it is, no other objectives or questions:\n\n{chunk}"

                summary = run_ollama1(prompt)
                if summary:
                    summaries.append(summary)
                else:
                    print(f"⚠️ Failed to summarize chunk {idx}")

        combined = "".join(summaries)
        if len(combined) > max_length:

            print("summarizing again")
            chunks = textwrap.wrap(combined, width=2000)

            summaries = []
            for idx, chunk in enumerate(chunks):
                prompt = f"Summarize shortly make shorter than it is. based on {prompt}, keep all details and facts, u are not doing anything else but summarizing/consolidating, return it back do not tell me how good it is, no other objectives or questions:\n\n{chunk}"

                summary = run_ollama1(prompt)
                if summary:
                    summaries.append(summary)
                else:
                    print(f"⚠️ Failed to summarize chunk {idx}")

        combined = "\n".join(summaries)
        prompt = f"You work for nearestdoor.com. This is scraped info from the internet based on {prompt}, keep all info/details, keep all information/facts/details operating hours phone website ect and turn it into an overview, no other objectives or questions, it is 2025:\n\n{combined}"

        summary = run_ollama4(prompt)
        if check_if_aggregate_is_good(prompt,summary):
            return True, summary
        else:
            return False, None

    except Exception as e:
        print(f"❌ Error in summarize_large_text: {e}")
        return False, None
def create_aggregate_plan(aggregate: str) -> bool:
    try:
        prompt = (
            "Determine if this information is enough to generate an 'article', 'faq', 'history' snippet.\n\n"
            f"{aggregate}\n\n"
            "Reply only with the names of sections that can be created: 'article', 'faq', 'history'.\n "
            "Do not include a section name if there isn't enough information to generate that section."
        )

        response = run_ollama4(prompt)
        response_clean = response.strip().lower()

        valid = {"article", "faq", "history"}
        sections = [s for s in valid if s in response_clean]

        return True, sections if sections else []

    except Exception as e:
        print(f"❌ Error checking aggregate: {e}")
        return False, None
def check_if_aggregate_is_good(prompt, aggregate: str) -> bool:
    try:
        prompt = (
            f"You are validating an overview of scraped content from multiple sources based on shop {prompt}.\n"
            f"{aggregate}\n\n"
            f"Reply only `true` or `false` — is the summary solid and contains useful facts and is about {prompt}? "
        )
        result = run_ollama4(prompt).strip().lower()
        return "true" in result
    except Exception as e:
        print(f"❌ Error checking aggregate: {e}")
        return False
def generate_article_faq_history(prompt, aggregate: str, approved_sections: list[str]) -> dict | None:
    def generate_and_validate(section_name: str, section_prompt: str) -> str | None:
        raw = run_ollama4(section_prompt).strip()
        if validate_section_html(prompt, section_name, raw):
            return raw
        print(f"⚠️ {section_name} section failed validation, attempting fix.")
        fixed = fix_section_html(prompt, section_name, raw)
        if fixed and validate_section_html(prompt, section_name, fixed):
            return fixed
        print(f"❌ {section_name} section discarded after failed validation and fix.")
        return None
    try:
        results = {}
        plan = f"You will get a summary and write for consumers to read as a nearestdoor.com writer reporting on {prompt}, remove bad info and random questions or info, remove stuff that doesnt make sense, DO NOT INCLUDE EXTRA TEXT, PREFIXES, MARKS, QUOTES OR COMMENTS, ONLY GIVE THE WANTED TEXT NOTHING ELSE, NO HTML, use key words, make it thorough, you work for nearestdoor.com as our writer about {prompt}, you report on {prompt} for nearestdoor.com. must be just text nothing before or after, DO NOT USE ** or *, no asterisks\n"
        if "article" in approved_sections:
            article_prompt = (
                f"Using the following plan:\n\n{plan}\n\n"
                f"And the following content:\n\n{aggregate}\n\n"
                f"Write a detailed, SEO-friendly article about the {prompt}."
            )
            article = generate_and_validate("article", article_prompt)
            if article:
                results["article"] = article

        if "faq" in approved_sections:
            faq_prompt = (
                f"Using the following plan:\n\n{plan}\n\n"
                f"And the following content:\n\n{aggregate}\n\n"
                f"Generate a detailed and useful FAQ section for the {prompt}."
            )
            faq = generate_and_validate("faq", faq_prompt)
            if faq:
                results["faq"] = faq


        if "history" in approved_sections:
            history_prompt = (
                f"Using the following plan:\n\n{plan}\n\n"
                f"And the following content:\n\n{aggregate}\n\n"
                f"Write a backstory/history overview of the {prompt}:"
            )
            history = generate_and_validate("history", history_prompt)
            if history:
                results["history"] = history

        return True, results if results else None

    except Exception as e:
        print(f"❌ Error generating article/faq/history: {e}")
        return False, None


import json
import re
def find_available_fields(aggregate):
    try:
        print('finding available fields', aggregate)
        field_list = list(FIELD_EXTRACTORS.keys())
        field_str = ', '.join(field_list)

        prompt = (
            f"The following content is scraped and summarized:\n\n{aggregate}\n\n"
            f"Which of the following fields can be confidently extracted from it?\n"
            f"{field_str}\n\n"
            "Only return a list of field keys that are clearly available.\n"
            "If a key is not clearly extractable from the content, do not include it in response."
        )

        raw_response = run_ollama4(prompt).strip()
        print(f"🧠 Raw LLM response:\n{raw_response}")

        # Normalize and clean up the response text
        response_clean = raw_response.lower()

        # Now just match by substring presence (don't trust format)
        detected = [field for field in field_list if field.lower() in response_clean]

        # Always include extract_categories (can always try to infer)
        if "extract_categories" not in detected:
            detected.append("extract_categories")

        return True, detected

    except Exception as e:
        print(f"❌ Error detecting available fields: {e}")
        return False, None


def extract_clean_json_structure(text):
    try:
        print("extract got", text)
        match = re.search(r"(|\{.*?\})", text.strip(), re.DOTALL)
        if not match:
            return None
        raw = match.group(0)
        json_ready = raw.replace("'", '"')
        return json.loads(json_ready)  # ✅ Returns a real dict or list
    except Exception as e:
        print(f"⚠️ extract_clean_json_structure failed: {e}")
        return None



def extract_fields_from_aggregate(aggregate: str, available_fields: list[str]) -> dict | None:
    try:
        extracted = {}

        for field in available_fields:
            prompt = FIELD_EXTRACTORS[field] + "\n\n" + aggregate
            raw_value = run_ollama4(prompt).strip()

            # Clean BEFORE validating
            cleaned_value = extract_clean_json_structure(raw_value) or raw_value

            validator = FIELD_VALIDATORS.get(field)
            is_valid = validator(cleaned_value) if validator else True

            # If fails local validation, use LLM validator
            if not is_valid:
                is_valid = check_with_ollama_format_validator(field, cleaned_value)

            # If still invalid, try to fix it via LLM
            if not is_valid:
                print(f"⚠️ Invalid format for {field}, attempting fix.")
                fixed = attempt_fix_with_ollama(field, cleaned_value)

                if fixed and validator(fixed):
                    is_valid = check_with_ollama_format_validator(field, fixed)
                    if is_valid:
                        extracted[field] = fixed
                    continue

                print(f"❌ Discarding {field}, failed validation and repair.")
                continue

            extracted[field] = cleaned_value
            print("EXTRACTED")
            print(cleaned_value)

        return True, extracted if extracted else None

    except Exception as e:
        print(f"❌ Error extracting fields: {e}")
        return False, None

def validate_section_html(prompt,section: str, html: str) -> bool:
    try:
        prompt = (
              "Reply only with `true` or `false`.\n"
            f"You're validating a block of text meant for a '{section}' section about {prompt} for nearestdoor.com.\n\n"
            f"{html}\n\n"
            "No ** or *, no asterisks\n"
            f"does the info seem valid and real?\n"

            "Article must be written as nearestdoor.com writer\n "
            "Does the info contain random questions or off topic parts?\n"
            " Does it make sense and are there any comments or prefixes before or after the text? It is written as a nearestdoor.com writer reporting on a shop? must be only the text wanted, nothing before or after\n"
            "Reply only with `true` or `false`.\n"

        )
        result = run_ollama4(prompt).strip().lower()
        return "true" in result
    except Exception as e:
        print(f"❌ HTML validation failed for section {section}: {e}")
        return False

def check_with_ollama_format_validator(field_name: str, value: str) -> bool:
    try:
        prompt = (
              "Reply only with `true` or `false`.\n"
            f"You're validating the format of this field: `{field_name}`.\n\n"
            f"Content:\n{value}\n\n"

            f"does the info seem valid and real?\n"
            "Is the format correct for its expected structure (e.g. phone, JSON, email) and has no comments, quotes, marks or prefixes? nothing before or after json, must be valid json, no '', no ` \n "
            "will this parse correctly to json?\n"
            "Reply only with `true` or `false`.\n"
        )
        result = run_ollama4(prompt).strip().lower()
        return "true" in result
    except Exception as e:
        print(f"❌ Ollama format check failed for {field_name}: {e}")
        return False
def attempt_fix_with_ollama(field_name: str, value: str) -> str | None:
    try:
        fix_prompt = (
            f"The following field is not formatted correctly for `{field_name}`.\n\n"
            f"Broken value:\n{value}\n\n"
             "Output must parse correctly to json.\n"
            f"Fix it to match the expected format based on this instruction:\n{FIELD_EXTRACTORS[field_name]}\n\n"
            "Only return the fixed value, no comments or explanation, no marks, quotes, or prefixes. nothing before or after json, must be valid json, no ''"
        )
        return run_ollama4(fix_prompt).strip()
    except Exception as e:
        print(f"❌ Ollama fix failed for {field_name}: {e}")
        return None
def fix_section_html(prompt, section: str, html: str) -> str | None:
    try:
        prompt = (
            f"The following {section} section text is incorrect or does not follow required formatting rules.\n\n"
            f"{html}\n\n"
            f"based on {prompt}.\n"
            "Article must be written as nearestdoor.com writer \n"
            "Fix it based on these rules:\n"
  "No ** or *, no asterisks\n"

            "must not contain random info or comments, weird info\n"
            "- Be SEO-friendly, concise, and structured.\n"
            "no '' \n"
            "Only text no html"
            "must be only the section nothing before or after\n"
            "Only return the corrected text. No extra commentary, quotes, prefixes, or marks."
        )
        return run_ollama4(prompt).strip()
    except Exception as e:
        print(f"❌ Failed to fix {section} section: {e}")
        return None


API_BASE = "https://www.nearestdoor.com"  # Replace with actual server URL
CLIENT_ID = "client001"
HEARTBEAT_INTERVAL = 60  # seconds
SHOP_FLOW_STATIC = [
    #all expect completed or failed responses and timeout in 4 mins so u will check in heartbeat every 1 min
    "search",#gives shopwebsite if any and shop name, calls google wiki, and shopwebsite lookup returns all data from summarize and images with alts base64 if so, first is_main, or none stop
  "aggregate"#sends data from search and shop data, sends back final summary or none stop
    "createplan",#sends final summary and shop name, expects "article", "faq", "history" in [] if good, or none
    "create", #sends final summary and the data [] from createplan and shop data, sends     results["history"] faq article, or none
    "generate_meta_tags_from_aggregate", #send aggregate and shop name, expect return {  "title": title "description": description or none  }
    "find_available_fields", #sends aggregate, expects detected = [field for field in FIELD_EXTRACTORS if field.lower() in response] return detected if detected else [] or None
    "extract_fields_from_aggregate",#sends aggreagate and result from find avilable fields, expects   extracted[field] = raw_value return extracted if extracted else None or none
    "fillintheshop",#if youve made it this far add content created to the shop variables
]
async def handle_task(task, crawler):
    try:
        if not task or not isinstance(task, dict):
            print("❌ Skipping empty or malformed task.")
            return

        task_id = task.get("task_id")
        task_type = task.get("task_type")

        if not task_id or not isinstance(task_id, int):
            print(f"❌ Invalid task_id: {task_id}")
            return
        if not task_type:
            print("❌ Missing task_type.")
            return

        print(f"▶️ Task: {task_type} ({task_id})")

        result,summary, mainstring, images, extractedfields, foundfields = False, None, None, None, None, None
        match task_type:
            case "search": result, mainstring, images = await search2(task, crawler)
            case "aggregate": result, summary = summarize_large_text(task['target'].get('name', ''),task['target'].get('largetext', '') )
            case "createplan": result,aggregateplan =  create_aggregate_plan(task['target'].get('summary', ''))
            case "create": result,createdinfo =  generate_article_faq_history(task['target'].get('name', ''), task['target'].get('summary', ''),task['target'].get('aggregateplan', ''))

            case "find_available_fields": result, foundfields = find_available_fields(task['target'].get('summary', ''))
            case "extract_fields_from_aggregate": result, extractedfields = extract_fields_from_aggregate(task['target'].get('summary', ''), task['target'].get('foundfields', ''))
        print(result, task_type, "extraxted",extractedfields,"found", foundfields)
        if result:
            print(f"📤 Submitting result for {task_type} ({task_id})")
            try:
                if task_type == 'search':
                    res = requests.post(f"{API_BASE}/submit/{task_id}", json={"status": "success", "mainstring": mainstring, "images": images, "client_id": CLIENT_ID})

                if task_type == 'aggregate':
                    if summary:
                        res = requests.post(f"{API_BASE}/submit/{task_id}", json={"status": "success", "summary": summary, "client_id": CLIENT_ID})
                    else:
                        print("nosummary")
                        res = requests.post(f"{API_BASE}/submit/{task_id}", json={"status": "fail", "client_id": CLIENT_ID})


                if task_type == 'createplan':
                    res = requests.post(f"{API_BASE}/submit/{task_id}", json={"status": "success", "aggregateplan": aggregateplan, "client_id": CLIENT_ID})
                if task_type == 'create':
                    res = requests.post(f"{API_BASE}/submit/{task_id}", json={"status": "success", "createdinfo":createdinfo, "client_id": CLIENT_ID})
                if task_type == 'find_available_fields':
                    res = requests.post(f"{API_BASE}/submit/{task_id}", json={"status": "success", "foundfields": foundfields, "client_id": CLIENT_ID})
                if task_type == 'extract_fields_from_aggregate':
                    res = requests.post(f"{API_BASE}/submit/{task_id}", json={"status": "success", "extractedfields": extractedfields, "client_id": CLIENT_ID})

                print(f"Server responded: {res.status_code} - {res.text}")
                if res.status_code == 200:
                    print(f"✅ Submitted: {task_type}")
                else:
                    print(f"❌ Submit failed: {task_type} - {res.status_code}")
            except Exception as e:
                print(f"Submit exception: {e}")
        else:

            print(f"Submit Failure {task_type}")
            res = requests.post(f"{API_BASE}/submit/{task_id}", json={"status": "fail", "client_id": CLIENT_ID})


    except Exception as e:
        print(f"Task FAILED, Task: {task_type} ({task_id} {e}) ")
        return None
def get_task():
    try:
        print("📡 Polling for task...")
        r = requests.get(f"{API_BASE}/next-task", params={"client_id": CLIENT_ID}, timeout=30)
        print(f"API GET response: {r.status_code} - {r.text}")

        if r.status_code == 200:
            try:
                task = r.json()
                if not isinstance(task, dict) or "task_id" not in task:
                    print(f"⚠️ Invalid task structure received: {task}")
                    return None
                return task
            except Exception as e:
                print(f"❌ Failed to parse JSON: {e}")
                return None

        elif r.status_code == 204:
            print("⏳ No available task.")
            return None

        print(f"❌ Unexpected status code: {r.status_code}")
        return None
    except Exception as e:
        print(f"❌ Task fetch failed: {e}")
        return None

def poll_heartbeat(current_task_id=None):
    try:
        data = {"client_id": CLIENT_ID}
        if current_task_id:
            data["task_id"] = current_task_id
        requests.post(f"{API_BASE}/heartbeat", json=data)
        print("🫀 Heartbeat sent.")
    except Exception as e:
        print(f"⚠️ Failed heartbeat: {e}")

async def main():
    crawler_mgr = CrawlerManager()
    await crawler_mgr.start()

    last_heartbeat = 0

    try:
        while True:
            task = get_task()
            if task:
                now = time.time()
                if now - last_heartbeat > HEARTBEAT_INTERVAL:
                    poll_heartbeat(task.get("task_id"))
                    last_heartbeat = now
                await handle_task(task, crawler_mgr)
            else:
                print("⏳ No task available, waiting...")
                await asyncio.sleep(10)
    finally:
        await crawler_mgr.stop()
import nest_asyncio
import asyncio

nest_asyncio.apply()  # Allows re-entry of the running loop in Colab

# Then just run the main function like this
await main()


[INIT].... → Crawl4AI 0.5.0.post8
📡 Polling for task...
API GET response: 200 - {"task_id": 12713, "task_type": "search", "object_type": "shop", "data": {}, "target": {"id": 29994, "name": "dog-parkRotary Dog Park (In Cranberry Communtiy Park) Cranberry Pennsylvania", "website": null, "slug": "rotary-dog-park-in-cranberry-communtiy-park"}}
🫀 Heartbeat sent.
▶️ Task: search (12713)
getting google results
dog-parkRotary Dog Park (In Cranberry Communtiy Park) Cranberry Pennsylvania
🔍 running ollama for relevance check
🧠 Running Ollama with model=gemma3:1b
Ollama stdout:
False.

The title explicitly states it’s about “Rotary Dog Park (In Cranberry Communtiy Park) Cranberry Pennsylvania”.
🔍 running ollama for relevance check
🧠 Running Ollama with model=gemma3:1b
Ollama stdout:
False.
🔍 running ollama for relevance check
🧠 Running Ollama with model=gemma3:1b
Ollama stdout:
True.
✅ Relevant: https://www.cranberry4fun.com/Facilities/Facility/Details/Community-Park-36
[FETCH]... ↓ https://www.c